In [1]:
import pandas as pd
#import matplotlib.ticker as mtick
import numpy as np
from arcgis.gis import *
gis = GIS()

strTAZCB        = "TAZ_CB"
strTAZCBRes     = "TAZ_CB_Res"
strTAZLayerName = "USTM_TAZ_2021_09_22"
strCBLayerName  = "tl_2020_49_tabblock20_UTM"
strHUILayerName = "residential_parcels_pts"
strGEOIDField   = 'GEOID20'

# get project and map
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Map")[0]

# dissolve TAZs with < 4000 area into adjacent
areaThreshold = 4000

In [2]:
# set layer definition for TAZ
for lyr in m.listLayers(strTAZLayerName):
    lyrTAZ = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = "SUBAREAID = 1 AND CO_FIPS IN (49,35,11,57)"
        display('TAZ set to WF Area excluding Box Elder')
                        
                        
# set layer definition for TAZ
for lyr in m.listLayers(strCBLayerName):
    lyrCB = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = "COUNTYFP20 IN ('049','035','011','057')"
        display('Census Blocks set to only 4 WF counties')
        
for lyr in m.listLayers(strHUILayerName):
    lyrHUI = lyr


'TAZ set to WF Area excluding Box Elder'

'Census Blocks set to only 4 WF counties'

In [3]:
initial_count = arcpy.management.GetCount(strTAZLayerName)
print (initial_count)

# union with self to see if overlapping polygons
arcpy.analysis.Union([strTAZLayerName, strTAZLayerName], "tempTAZSelfUnion")
selfunion_count = arcpy.management.GetCount("tempTAZSelfUnion")
print (selfunion_count)

arcpy.management.Delete("tempTAZSelfUnion")

3393
3393


<Result 'true'>

In [4]:
initial_count = arcpy.management.GetCount(strCBLayerName)
print (initial_count)

# union with self to see if overlapping polygons
arcpy.analysis.Union([strCBLayerName, strCBLayerName], "tempCBSelfUnion")
selfunion_count = arcpy.management.GetCount("tempCBSelfUnion")
print (selfunion_count)

arcpy.management.Delete("tempCBSelfUnion")

71207
71207


<Result 'true'>

In [4]:
#union
arcpy.analysis.Union([strTAZLayerName, strCBLayerName], strTAZCB + "_temp")

# set layer definition for TAZ
for lyr in m.listLayers(strTAZCB + "_temp"):
    lyrTAZ = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = "FID_USTM_TAZ_2021_09_22 <> -1"
        display('TAZ set to WF Area excluding Box Elder')

arcpy.management.CopyFeatures(strTAZCB + "_temp", strTAZCB)
        
#hide layers
lyrTAZ.visible = False
lyrCB.visible = False

'TAZ set to WF Area excluding Box Elder'

In [10]:
# delete unneeded fields
fldlist = arcpy.ListFields(strTAZCB)

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','CO_TAZID',strGEOIDField,'Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management(strTAZCB,[fld.name])

print("DONE!")

deleting field: FID_USTM_TAZ_2021_09_22
deleting field: TAZID
deleting field: SA_TAZID
deleting field: CO_IDX
deleting field: SUBAREAID
deleting field: ACRES
deleting field: DEVACRES
deleting field: DEVPBLEPCT
deleting field: X
deleting field: Y
deleting field: ADJ_XY
deleting field: CO_FIPS
deleting field: CO_NAME
deleting field: CITY_NAME
deleting field: DISTSUPER
deleting field: DSUP_NAME
deleting field: DISTLRG
deleting field: DLRG_NAME
deleting field: DISTMED
deleting field: DMED_NAME
deleting field: DISTSML
deleting field: DSML_NAME
deleting field: SORT
deleting field: TAZID_V21B
deleting field: SATAZ_V21B
deleting field: COTAZ_V21B
deleting field: SUBID_V21B
deleting field: TAZID_V21A
deleting field: SATAZ_V21A
deleting field: COTAZ_V21A
deleting field: SUBID_V21A
deleting field: FID_tl_2020_49_tabblock20_UTM
deleting field: STATEFP20
deleting field: COUNTYFP20
deleting field: TRACTCE20
deleting field: BLOCKCE20
deleting field: NAME20
deleting field: MTFCC20
deleting field: UR20

In [11]:
arcpy.analysis.SpatialJoin(strHUILayerName, strTAZCB, strTAZCBRes, "JOIN_ONE_TO_MANY", "KEEP_ALL")
lyrHUI.visible = False

# delte 
fldlist = arcpy.ListFields(strTAZCBRes)

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','CO_TAZID',strGEOIDField,'residential_units','year_built','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management(strTAZCBRes,[fld.name])
print("DONE!")

deleting field: Join_Count
deleting field: TARGET_FID
deleting field: JOIN_FID
deleting field: CO_NAME
deleting field: COUNTY_ID
deleting field: PARCEL_ID
deleting field: TOT_VALUE
deleting field: land_value
deleting field: building_sqft
deleting field: FLOORS_CNT
deleting field: EFFBUILT_YR
deleting field: IS_OUG
deleting field: max_dua
deleting field: max_far
deleting field: max_height
deleting field: type1
deleting field: type2
deleting field: type3
deleting field: type4
deleting field: type5
deleting field: type6
deleting field: type7
deleting field: type8
deleting field: agriculture
deleting field: basebldg
deleting field: NoBuild
deleting field: redev_friction
deleting field: building_type_id
deleting field: x
deleting field: y
deleting field: parcel_id_REMM
deleting field: parcel_acres
deleting field: Split
deleting field: Split_Factor
deleting field: parcel_id_REMM_old
deleting field: building_type
deleting field: residential_price
deleting field: non_residential_price
deleting

In [7]:
# read in TAZ_CB to dataframe to more easily manipulate with Pandas
sdfTazCb = pd.DataFrame.spatial.from_featureclass("TAZ_CB")


In [8]:
# group and get all blocks with at least two 
sdfTazCb_gt1 = sdfTazCb.groupby([strGEOIDField],as_index=False).agg(COUNT=("CO_TAZID","count"))
sdfTazCb_gt1 = sdfTazCb_gt1[(sdfTazCb_gt1['COUNT']>1)]
sdfTazCb_gt1

,GEOID20,COUNT


In [3]:
for lyr in m.listLayers(strTAZCB):
    lyrTAZCB = lyr

In [8]:
interval = 1000
iFirst = 69000
iLast  = 74000

# toggle for import... since it's such a beast and you don't want to accidently run it
bRunImport = false

if bRunImport:

    for i in range(iFirst,iLast,interval):

        bFirst = True
        #create empty layer for appending
        lyrTAZCB.definitionQuery = strGEOIDField + " = '' AND CO_TAZID = 0"
        arcpy.management.CopyFeatures(strTAZCB, "TAZ_CB_Eliminated_" + str(i))


        for index, row in sdfTazCb_gt1.iterrows():

            if index>=i and index<i+interval: 
                if bFirst:
                    print(i)
                bFirst = False

                # create layer with only splits from a given census block so that eliminates area always back to original census block and not an adjancent
                lyrTAZCB.definitionQuery = strGEOIDField + " = '" + row[strGEOIDField] + "'"

                print (index, row[strGEOIDField])

                arcpy.management.CopyFeatures(strTAZCB, "_temp")

                #select and eliminate pieces with area less than threshold
                arcpy.management.SelectLayerByAttribute("_temp", "NEW_SELECTION", "Shape_Area < " + str(areaThreshold))

                arcpy.management.Eliminate("_temp", "_temp_eliminated", "LENGTH")

                arcpy.management.Merge("TAZ_CB_Eliminated_" + str(i) +";_temp_eliminated", r"_temp_merged", 'CO_TAZID "CO_TAZID" true true false 4 Long 0 0,First,#,TAZ_CB_Eliminated_" + str(i) + ",CO_TAZID,-1,-1,_temp_eliminated,CO_TAZID,-1,-1;GEOID20 "GEOID20" true true false 15 Text 0 0,First,#,TAZ_CB_Eliminated_" + str(i) + ",GEOID20,0,15,_temp_eliminated,GEOID20,0,15;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,TAZ_CB_Eliminated_" + str(i) + ",Shape_Length,-1,-1,_temp_eliminated,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,TAZ_CB_Eliminated_" + str(i) + ",Shape_Area,-1,-1,_temp_eliminated,Shape_Area,-1,-1', "NO_SOURCE_INFO")

                arcpy.management.Delete("TAZ_CB_Eliminated_" + str(i))
                
                arcpy.management.CopyFeatures("_temp_merged", "TAZ_CB_Eliminated_" + str(i))
                
                arcpy.management.Delete("_temp_merged")

69000
69000 490572103042002
69001 490572103042003
69004 490572103042006
69005 490572103042007
69025 490572103042027
69026 490572103042028
69035 490572103043000
69039 490572103043004
69040 490572103043005
69041 490572103043006
69042 490572103043007
69043 490572103043008
69045 490572103043010
69046 490572103043011
69049 490572103043014
69050 490572103043015
69059 490572103043024
69060 490572103043025
69062 490572103043027
69064 490572103043029
69066 490572103043031
69068 490572103051001
69080 490572103051013
69086 490572103051019
69087 490572103051020
69088 490572103052000
69090 490572103052002
69093 490572103052005
69094 490572103052006
69096 490572103052008
69097 490572103052009
69099 490572103052011
69102 490572103052014
69103 490572103052015
69105 490572103052017
69109 490572103052021
69110 490572103052022
69113 490572103052025
69115 490572103052027
69116 490572103052028
69117 490572103052029
69119 490572103052031
69121 490572103052033
69123 490572103053001
69127 490572103053005
6913

70376 490572105182024
70381 490572105182029
70382 490572105182030
70386 490572105182034
70388 490572105182036
70389 490572105182037
70390 490572105182038
70391 490572105182039
70392 490572105182040
70393 490572105182041
70394 490572105182042
70395 490572105182043
70400 490572105182048
70404 490572105182052
70408 490572105182056
70409 490572105182057
70410 490572105182058
70411 490572105182059
70412 490572105182060
70414 490572105182062
70416 490572105182064
70417 490572105182065
70418 490572105182066
70419 490572105182067
70424 490572105182072
70425 490572105182073
70426 490572106001000
70427 490572106001001
70432 490572106001006
70433 490572106001007
70437 490572106001011
70446 490572106001020
70447 490572106001021
70451 490572106001025
70452 490572106001026
70453 490572106002000
70454 490572106002001
70456 490572106002003
70458 490572106002005
70461 490572106002008
70464 490572106002011
70465 490572106002012
70467 490572106002014
70468 490572106002015
70469 490572106002016
70474 4905

In [5]:
# get features with no splits
arcpy.analysis.PairwiseDissolve(strTAZCB, "TAZ_CB_Dissolve", strGEOIDField, "CO_TAZID MIN;CO_TAZID COUNT", "MULTI_PART")

for lyr in m.listLayers("TAZ_CB_Dissolve"):
    lyrTAZCBDissolve = lyr
lyrTAZCBDissolve.definitionQuery = "COUNT_CO_TAZID = 1"

arcpy.management.CopyFeatures("TAZ_CB_Dissolve", "TAZ_CB_Eliminated_NoSplits")
arcpy.management.Delete("TAZ_CB_Dissolve")

<Result 'true'>

In [7]:
# set layer definition for TAZ
for lyr in m.listLayers(strTAZLayerName):
    lyrTAZ = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = "CO_FIPS IN (49,35,11,57)"
        display('TAZ set to WF Area excluding Box Elder')
                        

            

'TAZ set to WF Area excluding Box Elder'

In [17]:
#for lyr in m.listLayers():
#    print(lyr.name)

import arcgisscripting, os
gp = arcgisscripting.create(9.3)

gp.workspace = 'Census Block to TAZ.gdb'
fcs = gp.ListFeatureClasses()

strAll = ""

for fc in fcs:
    if "CB_Eliminated" in fc:
        strAll += fc + ";"
        print (fc)

print(strAll)

TAZ_CB_Eliminated_0
TAZ_CB_Eliminated_2000
TAZ_CB_Eliminated_4000
TAZ_CB_Eliminated_6000
TAZ_CB_Eliminated_8000
TAZ_CB_Eliminated_10000
TAZ_CB_Eliminated_12000
TAZ_CB_Eliminated_14000
TAZ_CB_Eliminated_16000
TAZ_CB_Eliminated_18000
TAZ_CB_Eliminated_20000
TAZ_CB_Eliminated_22000
TAZ_CB_Eliminated_24000
TAZ_CB_Eliminated_26000
TAZ_CB_Eliminated_28000
TAZ_CB_Eliminated_30000
TAZ_CB_Eliminated_32000
TAZ_CB_Eliminated_34000
TAZ_CB_Eliminated_36000
TAZ_CB_Eliminated_37000
TAZ_CB_Eliminated_38000
TAZ_CB_Eliminated_39000
TAZ_CB_Eliminated_40000
TAZ_CB_Eliminated_41000
TAZ_CB_Eliminated_42000
TAZ_CB_Eliminated_43000
TAZ_CB_Eliminated_44000
TAZ_CB_Eliminated_45000
TAZ_CB_Eliminated_46000
TAZ_CB_Eliminated_47000
TAZ_CB_Eliminated_48000
TAZ_CB_Eliminated_49000
TAZ_CB_Eliminated_50000
TAZ_CB_Eliminated_51000
TAZ_CB_Eliminated_52000
TAZ_CB_Eliminated_53000
TAZ_CB_Eliminated_54000
TAZ_CB_Eliminated_55000
TAZ_CB_Eliminated_56000
TAZ_CB_Eliminated_57000
TAZ_CB_Eliminated_58000
TAZ_CB_Eliminated_59000


In [18]:
# merge all eliminated layers

arcpy.management.Merge(strAll, "TAZ_CB_Combined")

<Result 'Census Block to TAZ.gdb\\TAZ_CB_Combined'>

In [20]:
myFCls = "TAZ_CB_Combined"
arcpy.AddField_management(myFCls, "UNIQ_ID", "LONG")
dataDict = {}
i = 1
updateRows = arcpy.da.UpdateCursor(myFCls, ["OBJECTID", "UNIQ_ID"])
for updateRow in updateRows:
    if updateRow[0] not in dataDict:
        dataDict[updateRow[0]] = i
        i+=1
    updateRow[1] = dataDict[updateRow[0]]
    updateRows.updateRow(updateRow)
del updateRow, updateRows

In [21]:
# looks like eliminate function didn't carry over GEOID and TAZID fields correctly.... they are null
# so, create points in polygon and join back to TAZ_CB feature class

arcpy.management.FeatureToPoint("TAZ_CB_Combined", "TAZ_CB_Combined_FeatureToPoint", "INSIDE")

<Result 'Census Block to TAZ.gdb\\TAZ_CB_Combined_FeatureToPoint'>

In [10]:
# spatial join done manually to TAZ_CB_Combined_PointToOrig

# export to table
arcpy.conversion.TableToTable("TAZ_CB_Combined_FeatureToPoint_BackToTAZCB", "Census Block to TAZ.gdb", "TAZ_CB_Combined_FeatureToPoint_BackToTAZCB_Table")


<Result 'Census Block to TAZ.gdb\\TAZ_CB_Combined_FeatureToPoint_BackToTAZCB_Table'>

In [12]:
# join table back to polygons
# done manually since wasn't working
#arcpy.management.AddJoin("TAZ_CB_Combined", "UNIQ_ID", "TAZ_CB_Combined_FeatureToPoint_BackToTAZCB_Table", "UNIQ_ID")

arcpy.management.CopyFeatures("TAZ_CB_Combined", "TAZ_CB_Combined_withTAZGEOids")

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\Census Block to TAZ.gdb\\TAZ_CB_Combined_withTAZGEOids'>

In [14]:
# for missed ones found in Weber county
# manually selected and disolved TAZ_CB by GEOID20
sdfMissed = pd.DataFrame.spatial.from_featureclass("TAZ_CB_Missed_Dissolved")

# toggle for import... since it's such a beast and you don't want to accidently run it
bRunImport = True

if bRunImport:

    bFirst = True
    #create empty layer for appending
    lyrTAZCB.definitionQuery = strGEOIDField + " = '' AND CO_TAZID = 0"
    arcpy.management.CopyFeatures(strTAZCB, "TAZ_CB_Eliminated_Missed")

    for index, row in sdfMissed.iterrows():

        # create layer with only splits from a given census block so that eliminates area always back to original census block and not an adjancent
        lyrTAZCB.definitionQuery = strGEOIDField + " = '" + row[strGEOIDField] + "'"

        print (index, row[strGEOIDField])

        arcpy.management.CopyFeatures(strTAZCB, "_temp")

        #select and eliminate pieces with area less than threshold
        arcpy.management.SelectLayerByAttribute("_temp", "NEW_SELECTION", "Shape_Area < " + str(areaThreshold))

        arcpy.management.Eliminate("_temp", "_temp_eliminated", "LENGTH")

        arcpy.management.Merge("TAZ_CB_Eliminated_Missed;_temp_eliminated", r"_temp_merged", 'CO_TAZID "CO_TAZID" true true false 4 Long 0 0,First,#,TAZ_CB_Eliminated_Missed,CO_TAZID,-1,-1,_temp_eliminated,CO_TAZID,-1,-1;GEOID20 "GEOID20" true true false 15 Text 0 0,First,#,TAZ_CB_Eliminated_Missed,GEOID20,0,15,_temp_eliminated,GEOID20,0,15;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,TAZ_CB_Eliminated_Missed,Shape_Length,-1,-1,_temp_eliminated,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,TAZ_CB_Eliminated_Missed,Shape_Area,-1,-1,_temp_eliminated,Shape_Area,-1,-1', "NO_SOURCE_INFO")

        arcpy.management.Delete("TAZ_CB_Eliminated_Missed")

        arcpy.management.CopyFeatures("_temp_merged", "TAZ_CB_Eliminated_Missed")

        arcpy.management.Delete("_temp_merged")

0 
1 490572103031019
2 490572103031020
3 490572103031021
4 490572103031023
5 490572103031025
6 490572103031026
7 490572103031027
8 490572103031029
9 490572103031030
10 490572103031031
11 490572103031032
12 490572103031034
13 490572103031036
14 490572103031037
15 490572103031038
16 490572103031039
17 490572103032003
18 490572103032007
19 490572103032009
20 490572103032011
21 490572103032012
22 490572103032014
23 490572103032015
24 490572103032017
25 490572103032019
26 490572103032020
27 490572103032021
28 490572103032022
29 490572103032023
30 490572103032025
31 490572103033000
32 490572103033002
33 490572103033003
34 490572103033004
35 490572103033005
36 490572103033006
37 490572103033007
38 490572103033008
39 490572103033009
40 490572103033010
41 490572103033015
42 490572103033016
43 490572103033018
44 490572103033019
45 490572103033020
46 490572103033021
47 490572103034001
48 490572103034002
49 490572103034003
50 490572103034004
51 490572103034005
52 490572103034008
53 490572103034009

In [15]:
# merge all eliminated layers

arcpy.management.Merge("TAZ_CB_Eliminated_Missed;TAZ_CB_Combined_withTAZGEOids", "TAZ_CB_Combined_withMissed")

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\Census Block to TAZ.gdb\\TAZ_CB_Combined_withMissed'>

In [ ]:
# manullay rename TAZ_CB_Combined to TAZ_CB_Combined_OLD and TAZ_CB_Combined_withMissed to TAZ_CB_Combined

In [16]:
#replace nulls with zeros
fieldObs = arcpy.ListFields('TAZ_CB_Combined')  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor('TAZ_CB_Combined', fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
      
      
        curU.updateRow(rowU)

del curU




In [17]:
# # create new TAZID field and populate with correct value... adding the two fields, since one or other is always 0
# arcpy.management.AddField("TAZ_CB_Combined", "CO_TAZID_Combined", "INTEGER")
# arcpy.CalculateField_management("TAZ_CB_Combined", "CO_TAZID_Combined", "!MIN_CO_TAZID! + !CO_TAZID!", "PYTHON", "")
# 
# # add _shapearea field since sdf can't see Shape.Area field
# arcpy.management.AddField("TAZ_CB_Combined", "_shapearea", "DOUBLE")
# arcpy.CalculateField_management("TAZ_CB_Combined", "_shapearea", "!shape.area!", "PYTHON", "")
# 
# # delete unneeded fields
# arcpy.management.DeleteField("TAZ_CB_Combined","MIN_CO_TAZID")
# arcpy.management.DeleteField("TAZ_CB_Combined","CO_TAZID")
# arcpy.management.DeleteField("TAZ_CB_Combined","COUNT_CO_TAZID")
# 
# # get back to CO_TAZID name
# arcpy.management.AddField("TAZ_CB_Combined", "CO_TAZID", "INTEGER")
# arcpy.CalculateField_management("TAZ_CB_Combined", "CO_TAZID", "!CO_TAZID_Combined!", "PYTHON", "")
# arcpy.management.DeleteField("TAZ_CB_Combined","CO_TAZID_Combined")

In [25]:
sdfTAZCB_Clean = pd.DataFrame.spatial.from_featureclass("TAZ_CB_Combined")
display(sdfTAZCB_Clean)

,OBJECTID,CO_TAZID,GEOID20,_shapearea,SHAPE
0,1,110118,490111251021000,2.189653e+05,"{""rings"": [[[424307.02049999963, 4548771.3862]..."
1,2,110322,490111251021000,4.427046e+06,"{""rings"": [[[428230.7879999997, 4551003.972300..."
2,3,110118,490111251021001,4.324561e+05,"{""rings"": [[[424302.21970000025, 4549643.15399..."
3,4,110322,490111251021001,4.376192e+05,"{""rings"": [[[424326.8783, 4551043.8178], [4243..."
4,5,110118,490111251021002,1.596237e+04,"{""rings"": [[[423631.1037999997, 4550951.9011],..."
...,...,...,...,...,...
36475,36514,570116,490572103041023,7.407797e+04,"{""rings"": [[[418913.7887000004, 4573306.1085],..."
36476,36515,570092,490572103042000,2.420393e+05,"{""rings"": [[[413963.74980000034, 4576631.8826]..."
36477,36516,570091,490572103042001,7.306347e+05,"{""rings"": [[[414483.13269999996, 4575284.56959..."
36478,36517,570099,490572103042001,1.049905e+06,"{""rings"": [[[414874, 4574460.199999999], [4148..."


In [32]:
# get rid of duplicate index
#if sdfTAZCB_Clean.index.is_unique==False:
#    sdfTAZCB_Clean = sdfTAZCB_Clean.reset_index()
#    sdfTAZCB_Clean = sdfTAZCB_Clean.drop(columns=('index'))
#    display(sdfTAZCB_Clean)

In [26]:
# create numbering of pieces base on a cumulative count
sdfTAZCB_Clean['PIECECNT'] = sdfTAZCB_Clean.groupby([strGEOIDField]).cumcount()+1
display(sdfTAZCB_Clean)

,OBJECTID,CO_TAZID,GEOID20,_shapearea,SHAPE,PIECECNT
0,1,110118,490111251021000,2.189653e+05,"{""rings"": [[[424307.02049999963, 4548771.3862]...",1
1,2,110322,490111251021000,4.427046e+06,"{""rings"": [[[428230.7879999997, 4551003.972300...",2
2,3,110118,490111251021001,4.324561e+05,"{""rings"": [[[424302.21970000025, 4549643.15399...",1
3,4,110322,490111251021001,4.376192e+05,"{""rings"": [[[424326.8783, 4551043.8178], [4243...",2
4,5,110118,490111251021002,1.596237e+04,"{""rings"": [[[423631.1037999997, 4550951.9011],...",1
...,...,...,...,...,...,...
36475,36514,570116,490572103041023,7.407797e+04,"{""rings"": [[[418913.7887000004, 4573306.1085],...",1
36476,36515,570092,490572103042000,2.420393e+05,"{""rings"": [[[413963.74980000034, 4576631.8826]...",1
36477,36516,570091,490572103042001,7.306347e+05,"{""rings"": [[[414483.13269999996, 4575284.56959...",1
36478,36517,570099,490572103042001,1.049905e+06,"{""rings"": [[[414874, 4574460.199999999], [4148...",2


In [30]:
sdfTAZCB_Clean.columns

Index(['OBJECTID', 'CO_TAZID', 'GEOID20', '_shapearea', 'SHAPE', 'PIECECNT'], dtype='object')

In [28]:
# for IDs determin which GEOIDS are not split
dfGEOID20s_NumPieces = sdfTAZCB_Clean.groupby([strGEOIDField],as_index=False).agg(NUMPIECES=('CO_TAZID','count'))
dfGEOID20s_NumPieces

,GEOID20,NUMPIECES
0,490111251021000,2
1,490111251021001,2
2,490111251021002,1
3,490111251021003,3
4,490111251021004,2
...,...,...
31773,490572112025022,1
31774,490572112025023,1
31775,490572112025024,1
31776,490572112025025,1


In [35]:
#dfGEOID10s_NumPieces[dfGEOID10s_NumPieces[strGEOIDField]=='490351128051003']

In [31]:
sdfTAZCB_withNumPieces = pd.DataFrame.merge(sdfTAZCB_Clean,dfGEOID20s_NumPieces,on=strGEOIDField,how='left')
sdfTAZCB_withNumPieces

,OBJECTID,CO_TAZID,GEOID20,_shapearea,SHAPE,PIECECNT,NUMPIECES
0,1,110118,490111251021000,2.189653e+05,"{'rings': [[[424307.02049999963, 4548771.3862]...",1,2
1,2,110322,490111251021000,4.427046e+06,"{'rings': [[[428230.7879999997, 4551003.972300...",2,2
2,3,110118,490111251021001,4.324561e+05,"{'rings': [[[424302.21970000025, 4549643.15399...",1,2
3,4,110322,490111251021001,4.376192e+05,"{'rings': [[[424326.8783, 4551043.8178], [4243...",2,2
4,5,110118,490111251021002,1.596237e+04,"{'rings': [[[423631.1037999997, 4550951.9011],...",1,1
...,...,...,...,...,...,...,...
36475,36514,570116,490572103041023,7.407797e+04,"{'rings': [[[418913.7887000004, 4573306.1085],...",1,1
36476,36515,570092,490572103042000,2.420393e+05,"{'rings': [[[413963.74980000034, 4576631.8826]...",1,1
36477,36516,570091,490572103042001,7.306347e+05,"{'rings': [[[414483.13269999996, 4575284.56959...",1,3
36478,36517,570099,490572103042001,1.049905e+06,"{'rings': [[[414874, 4574460.199999999], [4148...",2,3


In [32]:
# setup unique id which is combination of GEOID and the split number created when splitting by TAZs
sdfTAZCB_withNumPieces['BLOCKSPLITID'] = ""
sdfTAZCB_withNumPieces.loc[(sdfTAZCB_withNumPieces['NUMPIECES']==1), 'BLOCKSPLITID'] = sdfTAZCB_withNumPieces[strGEOIDField] + '_0'
sdfTAZCB_withNumPieces.loc[(sdfTAZCB_withNumPieces['NUMPIECES']!=1), 'BLOCKSPLITID'] = sdfTAZCB_withNumPieces[strGEOIDField] + '_' + sdfTAZCB_withNumPieces['PIECECNT'].astype(str)

display(sdfTAZCB_withNumPieces)
display(sdfTAZCB_withNumPieces[sdfTAZCB_withNumPieces[strGEOIDField]=='490351128051003'])

sdfTAZCB_withNumPieces['SMALL'] = 0
sdfTAZCB_withNumPieces.loc[sdfTAZCB_withNumPieces['_shapearea']<areaThreshold, 'SMALL'] = 1

#remove columns no longer needed
sdfTAZCB_Final = sdfTAZCB_withNumPieces[['BLOCKSPLITID',strGEOIDField,'CO_TAZID','SMALL','_shapearea']]

display(sdfTAZCB_Final)

,OBJECTID,CO_TAZID,GEOID20,_shapearea,SHAPE,PIECECNT,NUMPIECES,BLOCKSPLITID
0,1,110118,490111251021000,2.189653e+05,"{'rings': [[[424307.02049999963, 4548771.3862]...",1,2,490111251021000_1
1,2,110322,490111251021000,4.427046e+06,"{'rings': [[[428230.7879999997, 4551003.972300...",2,2,490111251021000_2
2,3,110118,490111251021001,4.324561e+05,"{'rings': [[[424302.21970000025, 4549643.15399...",1,2,490111251021001_1
3,4,110322,490111251021001,4.376192e+05,"{'rings': [[[424326.8783, 4551043.8178], [4243...",2,2,490111251021001_2
4,5,110118,490111251021002,1.596237e+04,"{'rings': [[[423631.1037999997, 4550951.9011],...",1,1,490111251021002_0
...,...,...,...,...,...,...,...,...
36475,36514,570116,490572103041023,7.407797e+04,"{'rings': [[[418913.7887000004, 4573306.1085],...",1,1,490572103041023_0
36476,36515,570092,490572103042000,2.420393e+05,"{'rings': [[[413963.74980000034, 4576631.8826]...",1,1,490572103042000_0
36477,36516,570091,490572103042001,7.306347e+05,"{'rings': [[[414483.13269999996, 4575284.56959...",1,3,490572103042001_1
36478,36517,570099,490572103042001,1.049905e+06,"{'rings': [[[414874, 4574460.199999999], [4148...",2,3,490572103042001_2


,OBJECTID,CO_TAZID,GEOID20,_shapearea,SHAPE,PIECECNT,NUMPIECES,BLOCKSPLITID
25166,25205,351033,490351128051003,12697.002869,"{'rings': [[[427117.3671000004, 4488352.2786],...",1,1,490351128051003_0


,BLOCKSPLITID,GEOID20,CO_TAZID,SMALL,_shapearea
0,490111251021000_1,490111251021000,110118,0,2.189653e+05
1,490111251021000_2,490111251021000,110322,0,4.427046e+06
2,490111251021001_1,490111251021001,110118,0,4.324561e+05
3,490111251021001_2,490111251021001,110322,0,4.376192e+05
4,490111251021002_0,490111251021002,110118,0,1.596237e+04
...,...,...,...,...,...
36475,490572103041023_0,490572103041023,570116,0,7.407797e+04
36476,490572103042000_0,490572103042000,570092,0,2.420393e+05
36477,490572103042001_1,490572103042001,570091,0,7.306347e+05
36478,490572103042001_2,490572103042001,570099,0,1.049905e+06


In [33]:
sdfTAZCB_Final[sdfTAZCB_Final[strGEOIDField]=='490351128051003']

,BLOCKSPLITID,GEOID20,CO_TAZID,SMALL,_shapearea
25166,490351128051003_0,490351128051003,351033,0,12697.002869


In [40]:
# SECOND ROUND


In [34]:
# group and get all blocks with at least two and one with area less than threshhold
sdfTazCb_gt1 = sdfTAZCB_Final.groupby([strGEOIDField],as_index=False).agg(COUNT=("CO_TAZID","count"),NUMSMALL=('SMALL','sum'))
sdfTazCb_gt1 = sdfTazCb_gt1[(sdfTazCb_gt1['COUNT']>1)]
sdfTazCb_gt1wSmall = sdfTazCb_gt1[(sdfTazCb_gt1['NUMSMALL']>0)]
sdfTazCb_gt1wSmall = sdfTazCb_gt1wSmall[sdfTazCb_gt1wSmall[strGEOIDField]!=""]
sdfTazCb_gt1wSmall

,GEOID20,COUNT,NUMSMALL
105,490111251032001,2,1
109,490111251032005,3,1
186,490111251041001,2,1
197,490111251041012,2,1
236,490111251042023,3,1
...,...,...,...
31499,490572110002010,2,1
31556,490572111003013,2,1
31576,490572111005000,2,2
31708,490572112023016,2,1


In [35]:
strCBS = "Census_Block_Splits"

arcpy.management.CopyFeatures("TAZ_CB_Combined", strCBS)

for lyr in m.listLayers(strCBS):
    lyrCBS = lyr


lyrCBS.definitionQuery = ""

In [52]:
# toggle for import... since it's such a beast
bRun = False


if bRun:
    
    lyrCBS.definitionQuery = strGEOIDField + " = 'X' AND CO_TAZID = 0"
    arcpy.management.CopyFeatures(strCBS, "TAZ_CBS_Eliminated")

    for index, row in sdfTazCb_gt1wSmall.iterrows():

        if index < 50000:
            # create layer with only splits from a given census block so that eliminates area always back to original census block and not an adjancent
            lyrCBS.definitionQuery = strGEOIDField + " = '" + row[strGEOIDField] + "'"

            print (index, row[strGEOIDField])

            arcpy.management.CopyFeatures(strCBS, "_temp")

            #select and eliminate pieces with area less than threshold
            arcpy.management.SelectLayerByAttribute("_temp", "NEW_SELECTION", "Shape_Area < " + str(areaThreshold))

            arcpy.management.Eliminate("_temp", "_temp_eliminated", "LENGTH")

            arcpy.management.Merge("TAZ_CBS_Eliminated;_temp_eliminated", r"_temp_merged", 'CO_TAZID "CO_TAZID" true true false 4 Long 0 0,First,#,TAZ_CBS_Eliminated,CO_TAZID,-1,-1,_temp_eliminated,CO_TAZID,-1,-1;GEOID20 "GEOID20" true true false 15 Text 0 0,First,#,TAZ_CBS_Eliminated,GEOID20,0,15,_temp_eliminated,GEOID20,0,15;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,TAZ_CBS_Eliminated,Shape_Length,-1,-1,_temp_eliminated,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,TAZ_CBS_Eliminated,Shape_Area,-1,-1,_temp_eliminated,Shape_Area,-1,-1', "NO_SOURCE_INFO")

            arcpy.management.Delete("TAZ_CBS_Eliminated")
            arcpy.management.CopyFeatures("_temp_merged", "TAZ_CBS_Eliminated")
            
arcpy.management.Delete("_temp")
arcpy.management.Delete("_temp_eliminated")
arcpy.management.Delete("_temp_merged")

105 490111251032001
109 490111251032005
186 490111251041001
197 490111251041012
236 490111251042023
332 490111253012001
378 490111253014001
397 490111253041000
424 490111253042010
455 490111253042041
456 490111253043000
495 490111253051022
526 490111253053010
537 490111253054009
550 490111253061000
577 490111253061027
630 490111253071021
636 490111253072000
642 490111253072006
647 490111253072011
668 490111253073017
685 490111253074009
689 490111253074013
711 490111253074035
747 490111254072012
763 490111254073006
779 490111254074005
791 490111254074017
804 490111254082000
872 490111254083024
948 490111254102025
1001 490111254112020
1033 490111254121008
1074 490111254131020
1144 490111254142009
1190 490111254151028
1229 490111255011019
1281 490111255013017
1345 490111255031007
1424 490111257012005
1428 490111257013000
1431 490111257013003
1456 490111257021005
1483 490111257022019
1556 490111258014004
1564 490111258014012
1583 490111258041000
1616 490111258041033
1657 490111258044000
17

19664 490490011031019
19666 490490011032000
19673 490490011032007
19708 490490011061000
19712 490490011062000
19762 490490011081000
19834 490490012012027
19873 490490012022011
19929 490490013003016
19973 490490014012000
19980 490490014012007
20114 490490015031000
20149 490490015032003
20189 490490015042000
20241 490490016012000
20278 490490017011006
20391 490490018021003
20399 490490018022003
20426 490490018031003
20444 490490019001000
20456 490490019002000
20458 490490019002002
20466 490490019003000
20491 490490020011016
20528 490490020022017
20532 490490021011000
20675 490490022043000
20698 490490022051007
20723 490490022052000
20770 490490022063000
20779 490490022081000
20780 490490022081001
20781 490490022081002
20827 490490022082022
20836 490490022091000
20842 490490022091006
20843 490490022092000
20847 490490022092004
20848 490490022092005
20855 490490022093000
20889 490490022101000
20935 490490022122002
20972 490490022123015
20980 490490022131003
20981 490490022131004
21029 4904

<Result 'true'>

In [36]:
strList = ""
for index, row in sdfTazCb_gt1wSmall.iterrows():
    strList = strList + "'" + row[strGEOIDField] + "',"
strList = "(" + strList[:-1] + ")"
strList

"('490111251032001','490111251032005','490111251041001','490111251041012','490111251042023','490111253012001','490111253014001','490111253041000','490111253042010','490111253042041','490111253043000','490111253051022','490111253053010','490111253054009','490111253061000','490111253061027','490111253071021','490111253072000','490111253072006','490111253072011','490111253073017','490111253074009','490111253074013','490111253074035','490111254072012','490111254073006','490111254074005','490111254074017','490111254082000','490111254083024','490111254102025','490111254112020','490111254121008','490111254131020','490111254142009','490111254151028','490111255011019','490111255013017','490111255031007','490111257012005','490111257013000','490111257013003','490111257021005','490111257022019','490111258014004','490111258014012','490111258041000','490111258041033','490111258044000','490111258052025','490111258071005','490111258072001','490111258093003','490111258102000','490111259053017','4901112

In [40]:
# toggle for import... since it's such a beast
bRunDelete = True

if bRunDelete:
    
    #strCBSEliminated = "TAZ_CBS_Eliminated"
    strCBSEliminated = "TAZ_CBS_Eliminated_SpatialJo"
    arcpy.management.CopyFeatures(strCBSEliminated, "_temp_CBS_Eliminated")

    lyrCBS.definitionQuery = ""
    arcpy.management.CopyFeatures(strCBS, "_temp_CBS_All")

    for lyr in m.listLayers("_temp_CBS_All"):
        lyrTempCBSAll = lyr
    
    # remove fields from data set to match original
    arcpy.DeleteField_management("_temp_CBS_All","BLOCKSPLITID")
    arcpy.DeleteField_management("_temp_CBS_All","SMALL"       )

    # filter by those that did not have another round of elimination
    sql = strGEOIDField + " NOT IN " + strList
    #print (sql)
    lyrTempCBSAll.definitionQuery = sql
    arcpy.management.CopyFeatures("_temp_CBS_All", "_temp_CBS_EliminatedRemoved")
   
    # remove fields from eliminated data set to match original - removed manually
    arcpy.DeleteField_management("_temp_CBS_Eliminated","BLOCKSPLITID")
    arcpy.DeleteField_management("_temp_CBS_Eliminated","SMALL"       )

    # modify this manually
    arcpy.management.Merge("_temp_CBS_EliminatedRemoved;_temp_CBS_Eliminated", r"Census_Block_Splits_Round2", 'GEOID20 "GEOID20" true true false 30 Text 0 0,First,#,_temp_CBS_EliminatedRemoved,GEOID20,0,30,_temp_CBS_Eliminated,GEOID20,0,15;CO_TAZID "CO_TAZID" true true false 4 Long 0 0,First,#,_temp_CBS_EliminatedRemoved,CO_TAZID,-1,-1,_temp_CBS_Eliminated,CO_TAZID,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,_temp_CBS_EliminatedRemoved,Shape_Length,-1,-1,_temp_CBS_Eliminated,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,_temp_CBS_EliminatedRemoved,Shape_Area,-1,-1,_temp_CBS_Eliminated,Shape_Area,-1,-1', "NO_SOURCE_INFO")

arcpy.management.Delete("_temp_CBS_Eliminated")
arcpy.management.Delete("_temp_CBS_All")
arcpy.management.Delete("_temp_CBS_EliminatedRemoved")



In [41]:
arcpy.management.AddField("Census_Block_Splits_Round2", "Acres", "DOUBLE")
arcpy.CalculateField_management("Census_Block_Splits_Round2", "Acres", "!shape.area@acres!", "PYTHON", "")

<Result 'Census_Block_Splits_Round2'>

In [3]:
# manually add back 490490105041104 and 490490104111183
# remove overlap 490111262041019 and 490351028021020
# manual edit to 490111262041019, 490490021021000,490490021022000,490490021021000  removing overlap
# CO_TAZID = 0

# CHECK NULL VALUES IN ROUND 2 FILE

In [4]:
# IDs Again

In [92]:
# CHECK OVERLAPPING POLYGONS

strLayerName = "Census_Block_Splits_Round2"

initial_count = arcpy.management.GetCount(strLayerName)
print (initial_count)

# union with self to see if overlapping polygons
arcpy.analysis.Union([strLayerName, strLayerName], "tempSelfUnion_" + strLayerName)
selfunion_count = arcpy.management.GetCount("tempSelfUnion_" + strLayerName)
print (selfunion_count)

#arcpy.management.Delete("tempSelfUnion_" + Census_Block_Splits_Round2)



35218
35208


In [93]:
# set layer definition for TAZ
for lyr in m.listLayers("tempSelfUnion_" + strLayerName):
    lyrAddress = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = "FID_Census_Block_Splits_Round2 <> FID_Census_Block_Splits_Round2_1"

In [95]:
# MANUALLY REMOVE OVERLAPS

# set layer definition for TAZ
for lyr in m.listLayers("tempSelfUnion_" + strLayerName):
    lyrAddress = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = ""

arcpy.management.CopyFeatures("tempSelfUnion_" + strLayerName, "Census_Block_Splits_Round3")

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\Census Block to TAZ.gdb\\Census_Block_Splits_Round3'>

In [97]:
arcpy.management.AddField("Census_Block_Splits_Round3", "PieceID", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Round3", "PieceID", "!OBJECTID!", "PYTHON", "")

<Result 'Census_Block_Splits_Round3'>

# SPLITS COMPLETE... NOW CALCULATIONS

In [123]:
strFinalTAZCBSplits = "Census_Block_Splits_Round3"

In [124]:
sdfTAZCB_Clean = pd.DataFrame.spatial.from_featureclass(strFinalTAZCBSplits)
display(sdfTAZCB_Clean)

,OBJECTID,GEOID20,CO_TAZID,Acres,GEOID20_1,Acres_1,PieceID,SHAPE
0,1,490111251021000,110118,54.107495,490111251021000,54.107495,1,"{""rings"": [[[424307.02049999963, 4548771.3862]..."
1,2,490111251021000,110322,1093.946855,490111251021000,1093.946855,2,"{""rings"": [[[428135.4545, 4550802.4669], [4281..."
2,3,490111251021001,110118,106.862239,490111251021001,106.862239,3,"{""rings"": [[[424302.21970000025, 4549643.15399..."
3,4,490111251021001,110322,108.138069,490111251021001,108.138069,4,"{""rings"": [[[424302.21970000025, 4549643.15399..."
4,5,490111251021002,110118,3.944387,490111251021002,3.944387,5,"{""rings"": [[[423631.1037999997, 4550951.9011],..."
...,...,...,...,...,...,...,...,...
35204,35205,490572111003013,570398,93.043177,490572111003013,93.043177,35205,"{""rings"": [[[416747.6473000003, 4557186.8597],..."
35205,35206,490572111005000,570391,0.986072,490572111005000,0.986072,35206,"{""rings"": [[[417508.43290000036, 4558374.2315]..."
35206,35207,490572112023016,570422,2.828907,490572112023016,2.828907,35207,"{""rings"": [[[421038.9335000003, 4554486.1985],..."
35207,35208,490572112023030,570422,6.180297,490572112023030,6.180297,35208,"{""rings"": [[[423440.20189999975, 4554125.6844]..."


In [125]:
# create numbering of pieces base on a cumulative count
sdfTAZCB_Clean['PIECECNT'] = sdfTAZCB_Clean.groupby([strGEOIDField]).cumcount()+1
display(sdfTAZCB_Clean)

,OBJECTID,GEOID20,CO_TAZID,Acres,GEOID20_1,Acres_1,PieceID,SHAPE,PIECECNT
0,1,490111251021000,110118,54.107495,490111251021000,54.107495,1,"{""rings"": [[[424307.02049999963, 4548771.3862]...",1
1,2,490111251021000,110322,1093.946855,490111251021000,1093.946855,2,"{""rings"": [[[428135.4545, 4550802.4669], [4281...",2
2,3,490111251021001,110118,106.862239,490111251021001,106.862239,3,"{""rings"": [[[424302.21970000025, 4549643.15399...",1
3,4,490111251021001,110322,108.138069,490111251021001,108.138069,4,"{""rings"": [[[424302.21970000025, 4549643.15399...",2
4,5,490111251021002,110118,3.944387,490111251021002,3.944387,5,"{""rings"": [[[423631.1037999997, 4550951.9011],...",1
...,...,...,...,...,...,...,...,...,...
35204,35205,490572111003013,570398,93.043177,490572111003013,93.043177,35205,"{""rings"": [[[416747.6473000003, 4557186.8597],...",1
35205,35206,490572111005000,570391,0.986072,490572111005000,0.986072,35206,"{""rings"": [[[417508.43290000036, 4558374.2315]...",1
35206,35207,490572112023016,570422,2.828907,490572112023016,2.828907,35207,"{""rings"": [[[421038.9335000003, 4554486.1985],...",1
35207,35208,490572112023030,570422,6.180297,490572112023030,6.180297,35208,"{""rings"": [[[423440.20189999975, 4554125.6844]...",1


In [126]:
# for IDs determin which GEOIDS are not split
dfGEOID20s_NumPieces = sdfTAZCB_Clean.groupby([strGEOIDField],as_index=False).agg(NUMPIECES=('CO_TAZID','count'))
dfGEOID20s_NumPieces

,GEOID20,NUMPIECES
0,490111251021000,2
1,490111251021001,2
2,490111251021002,1
3,490111251021003,3
4,490111251021004,2
...,...,...
31773,490572112025022,1
31774,490572112025023,1
31775,490572112025024,1
31776,490572112025025,1


In [127]:
sdfTAZCB_withNumPieces = pd.DataFrame.merge(sdfTAZCB_Clean,dfGEOID20s_NumPieces,on=strGEOIDField,how='left')
sdfTAZCB_withNumPieces

,OBJECTID,GEOID20,CO_TAZID,Acres,GEOID20_1,Acres_1,PieceID,SHAPE,PIECECNT,NUMPIECES
0,1,490111251021000,110118,54.107495,490111251021000,54.107495,1,"{'rings': [[[424307.02049999963, 4548771.3862]...",1,2
1,2,490111251021000,110322,1093.946855,490111251021000,1093.946855,2,"{'rings': [[[428135.4545, 4550802.4669], [4281...",2,2
2,3,490111251021001,110118,106.862239,490111251021001,106.862239,3,"{'rings': [[[424302.21970000025, 4549643.15399...",1,2
3,4,490111251021001,110322,108.138069,490111251021001,108.138069,4,"{'rings': [[[424302.21970000025, 4549643.15399...",2,2
4,5,490111251021002,110118,3.944387,490111251021002,3.944387,5,"{'rings': [[[423631.1037999997, 4550951.9011],...",1,1
...,...,...,...,...,...,...,...,...,...,...
35204,35205,490572111003013,570398,93.043177,490572111003013,93.043177,35205,"{'rings': [[[416747.6473000003, 4557186.8597],...",1,1
35205,35206,490572111005000,570391,0.986072,490572111005000,0.986072,35206,"{'rings': [[[417508.43290000036, 4558374.2315]...",1,1
35206,35207,490572112023016,570422,2.828907,490572112023016,2.828907,35207,"{'rings': [[[421038.9335000003, 4554486.1985],...",1,1
35207,35208,490572112023030,570422,6.180297,490572112023030,6.180297,35208,"{'rings': [[[423440.20189999975, 4554125.6844]...",1,1


In [128]:
# setup unique id which is combination of GEOID and the split number created when splitting by TAZs
sdfTAZCB_withNumPieces['BLOCKSPLITID'] = ""
sdfTAZCB_withNumPieces.loc[(sdfTAZCB_withNumPieces['NUMPIECES']==1), 'BLOCKSPLITID'] = sdfTAZCB_withNumPieces[strGEOIDField] + '_0'
sdfTAZCB_withNumPieces.loc[(sdfTAZCB_withNumPieces['NUMPIECES']!=1), 'BLOCKSPLITID'] = sdfTAZCB_withNumPieces[strGEOIDField] + '_' + sdfTAZCB_withNumPieces['PIECECNT'].astype(str)

#remove columns no longer needed
dfTAZCB_Final = sdfTAZCB_withNumPieces[['BLOCKSPLITID',strGEOIDField,'CO_TAZID','PieceID']]

display(dfTAZCB_Final)

,BLOCKSPLITID,GEOID20,CO_TAZID,PieceID
0,490111251021000_1,490111251021000,110118,1
1,490111251021000_2,490111251021000,110322,2
2,490111251021001_1,490111251021001,110118,3
3,490111251021001_2,490111251021001,110322,4
4,490111251021002_0,490111251021002,110118,5
...,...,...,...,...
35204,490572111003013_0,490572111003013,570398,35205
35205,490572111005000_0,490572111005000,570391,35206
35206,490572112023016_0,490572112023016,570422,35207
35207,490572112023030_0,490572112023030,570422,35208


In [129]:
#export to csv
dfTAZCB_Final.to_csv('dfTABCB_Final3.csv')

In [130]:
arcpy.management.AddJoin(strFinalTAZCBSplits, "PieceID", 'dfTABCB_Final3.csv', "PieceID")

<Result 'Census_Block_Splits_Round3'>

In [133]:
arcpy.management.CopyFeatures(strFinalTAZCBSplits, "Census_Block_Splits_Final")

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\Census Block to TAZ.gdb\\Census_Block_Splits_Final'>

In [134]:
arcpy.management.AddField("Census_Block_Splits_Final", "BLKSPLITID", "TEXT", field_length=25)
arcpy.CalculateField_management("Census_Block_Splits_Final", "BLKSPLITID", "!dfTABCB_Final3_csv_BLOCKSPLITID!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final", strGEOIDField, "TEXT", field_length=25)
arcpy.CalculateField_management("Census_Block_Splits_Final", strGEOIDField, "!" + strFinalTAZCBSplits + "_GEOID20!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final", "CO_TAZID", "LONG")
arcpy.CalculateField_management("Census_Block_Splits_Final", "CO_TAZID", "!" + strFinalTAZCBSplits + "_CO_TAZID!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final", "ACRES", "DOUBLE")
arcpy.CalculateField_management("Census_Block_Splits_Final", "ACRES", "!" + strFinalTAZCBSplits + "_Acres!", "PYTHON", "")


<Result 'Census_Block_Splits_Final'>

In [135]:
# delte 
fldlist = arcpy.ListFields("Census_Block_Splits_Final")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','CO_TAZID',strGEOIDField,'BLKSPLITID','Shape','Shape_Length','Shape_Area','ACRES']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("Census_Block_Splits_Final",[fld.name])
print("DONE!")

deleting field: Census_Block_Splits_Round3_GEOID20
deleting field: Census_Block_Splits_Round3_CO_TAZID
deleting field: Census_Block_Splits_Round3_Acres
deleting field: Census_Block_Splits_Round3_GEOID20_1
deleting field: Census_Block_Splits_Round3_Acres_1
deleting field: Census_Block_Splits_Round3_PieceID
deleting field: dfTABCB_Final3_csv_Field1
deleting field: dfTABCB_Final3_csv_BLOCKSPLITID
deleting field: dfTABCB_Final3_csv_GEOID20
deleting field: dfTABCB_Final3_csv_CO_TAZID
deleting field: dfTABCB_Final3_csv_PieceID
DONE!


In [136]:
bRun = False

if bRun:
    
    print("")
    
    #lyrHUI.definitionQuery = "year_built IS NULL Or year_built <= 2019"
    #arcpy.management.CopyFeatures(strHUILayerName, "ResUnits2019")   
    
    # LOOKS LIKE 2020 BUILT YEAR NOT COMPLETE, SINCE ONLY 54 LOCATIONS
    #lyrHUI.definitionQuery = "year_built IS NULL Or year_built <= 2020"
    #arcpy.management.CopyFeatures(strHUILayerName, "ResUnits2020")

In [137]:
strHUILayerName

'residential_parcels_pts'

In [138]:
arcpy.gapro.JoinFeatures("Census_Block_Splits_Final", "ResUnits2019", "CensusBlock_joined_ResUnits2019", "JOIN_ONE_TO_ONE", "CONTAINS", None, '', None, None, "residential_units SUM;residential_units COUNT", '', "KEEP_ALL")
#arcpy.gapro.JoinFeatures("Census_Block_Splits_Final", "ResUnits2020", "CensusBlock_joined_ResUnits2020", "JOIN_ONE_TO_ONE", "CONTAINS", None, '', None, None, "residential_units SUM;residential_units COUNT", '', "KEEP_ALL")

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\Census Block to TAZ.gdb\\CensusBlock_joined_ResUnits2019'>

In [139]:
# set layer definition for TAZ
for lyr in m.listLayers("AddressPoints_NAD83"):
    lyrAddress = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = "CountyID IN ('49011', '49035', '49049', '49057') And PtType NOT IN ('INDUSTRIAL', 'Industrial', 'COMMERCIAL', 'Commercial', 'Government', 'Vacant')"
        
arcpy.gapro.JoinFeatures("Census_Block_Splits_Final", "AddressPoints_NAD83", "Census_Block_Splits_Final_joined_AddressPoints", "JOIN_ONE_TO_ONE", "CONTAINS", None, '', None, None, "OBJECTID COUNT", '', "KEEP_ALL")

# NEED TO MANUALLY REMOVE BLOCKS WITHIN REMM AREA

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\Census Block to TAZ.gdb\\Census_Block_Splits_Final_joined_AddressPoints'>

In [140]:
# put into sdf to calculate distributions

#sdfResCBSplit2010 = pd.DataFrame.spatial.from_featureclass("CensusBlock_joined_ResUnits2010")
#dfResCBSplit2010 = sdfResCBSplit2010[['BLOCKSPLITID','GEOID10','ACRES','SUM_residential_units']]
#dfResCBSplit2010 = dfResCBSplit2010.rename(columns={'BLOCKSPLITID':'BLKSPLITID','SUM_residential_units':'HH10'})
#display(dfResCBSplit2010)

sdfResCBSplit2019 = pd.DataFrame.spatial.from_featureclass("CensusBlock_joined_ResUnits2019")
dfResCBSplit2019 = sdfResCBSplit2019[[strGEOIDField,'ACRES','BLKSPLITID','SUM_residential_units']]
dfResCBSplit2019 = dfResCBSplit2019.rename(columns={'BLKSPLITID':'BLKSPLITID','SUM_residential_units':'HH19'})
display(dfResCBSplit2019)

sdfAddCBSplit = pd.DataFrame.spatial.from_featureclass("Census_Block_Splits_Final_joined_AddressPoints")
dfAddCBSplit = sdfAddCBSplit[['BLKSPLITID','COUNT']]
dfAddCBSplit = dfAddCBSplit.rename(columns={'BLKSPLITID':'BLKSPLITID','COUNT':'ADDPT'})
display(dfAddCBSplit)


,GEOID20,ACRES,BLKSPLITID,HH19
0,490111269024010,3.402761,490111269024010_0,7.0
1,490111254152011,4.501047,490111254152011_2,15.0
2,490111270024054,0.315008,490111270024054_0,NaN
3,490111263033003,317.762203,490111263033003_0,NaN
4,490111264021004,122.763838,490111264021004_2,NaN
...,...,...,...,...
35204,490572110001005,2.584262,490572110001005_0,12.0
35205,490572107031007,8.197889,490572107031007_0,31.0
35206,490572106001017,3.022303,490572106001017_0,NaN
35207,490351152111046,59.018298,490351152111046_2,NaN


,BLKSPLITID,ADDPT
0,490490109002006_0,6.0
1,490490109003041_0,0.0
2,490490005071002_2,0.0
3,490572101011007_0,0.0
4,490351101063063_0,9.0
...,...,...
1581,490490102123009_2,0.0
1582,490351101063039_2,1.0
1583,490572104081052_1,0.0
1584,490490101074010_5,0.0


In [141]:
sdfAddCBSplit

,OBJECTID1,OBJECTID,BLKSPLITID,GEOID20,CO_TAZID,ACRES,COUNT,COUNT_OBJECTID,SHAPE
0,37,31640,490490109002006_0,490490109002006,491290,3.190031,6.0,6.0,"{""rings"": [[[451755.5696999999, 4467006.256899..."
1,54,31775,490490109003041_0,490490109003041,491328,84.684933,0.0,0.0,"{""rings"": [[[456796.3066999996, 4418573.942399..."
2,55,8670,490490005071002_2,490490005071002,491318,9.119740,0.0,0.0,"{""rings"": [[[440828.07880000025, 4467059.7184]..."
3,89,32450,490572101011007_0,490572101011007,570424,186.484851,0.0,0.0,"{""rings"": [[[421903.32859999966, 4578407.2665]..."
4,106,20610,490351101063063_0,490351101063063,351294,13.321258,9.0,9.0,"{""rings"": [[[435848.8942, 4491534.171599999], ..."
...,...,...,...,...,...,...,...,...,...
1581,35174,12439,490490102123009_2,490490102123009,491315,118.875443,0.0,0.0,"{""rings"": [[[428236.6830000002, 4478117.6778],..."
1582,35188,3534,490351101063039_2,490351101063039,351289,28.504496,1.0,1.0,"{""rings"": [[[449521.16509999987, 4497294.8345]..."
1583,35193,16014,490572104081052_1,490572104081052,570001,51.110099,0.0,0.0,"{""rings"": [[[394670.20249999966, 4567428.2447]..."
1584,35194,10854,490490101074010_5,490490101074010,491311,29.690670,0.0,0.0,"{""rings"": [[[403959.62040000036, 4461228.36659..."


In [142]:
dfResCBSplit2019

,GEOID20,ACRES,BLKSPLITID,HH19
0,490111269024010,3.402761,490111269024010_0,7.0
1,490111254152011,4.501047,490111254152011_2,15.0
2,490111270024054,0.315008,490111270024054_0,NaN
3,490111263033003,317.762203,490111263033003_0,NaN
4,490111264021004,122.763838,490111264021004_2,NaN
...,...,...,...,...
35204,490572110001005,2.584262,490572110001005_0,12.0
35205,490572107031007,8.197889,490572107031007_0,31.0
35206,490572106001017,3.022303,490572106001017_0,NaN
35207,490351152111046,59.018298,490351152111046_2,NaN


In [143]:
#dfResCBS    = pd.DataFrame.merge(dfResCBSplit2010, dfResCBSplit2019, on='BLKSPLITID')
#dfResCBSAdd = pd.DataFrame.merge(dfResCBS        , dfAddCBSplit    , on='BLKSPLITID', how='left')

dfResCBSAdd = pd.DataFrame.merge(dfResCBSplit2019, dfAddCBSplit    , on='BLKSPLITID', how='left')
dfResCBSAdd = dfResCBSAdd.fillna(0)

# add in address points
#dfResCBSAdd['HH10'] = dfResCBSAdd['HH10'] + dfResCBSAdd['ADDPT']
dfResCBSAdd['HH19'] = dfResCBSAdd['HH19'] + dfResCBSAdd['ADDPT']

#idx is the index of split, which is last number in BlockSplitID
dfResCBSAdd['SPLIT_IDX'] = dfResCBSAdd['BLKSPLITID'].str[-1:].astype(int)

dfResCBSAdd

,GEOID20,ACRES,BLKSPLITID,HH19,ADDPT,SPLIT_IDX
0,490111269024010,3.402761,490111269024010_0,7.0,0.0,0
1,490111254152011,4.501047,490111254152011_2,15.0,0.0,2
2,490111270024054,0.315008,490111270024054_0,0.0,0.0,0
3,490111263033003,317.762203,490111263033003_0,0.0,0.0,0
4,490111264021004,122.763838,490111264021004_2,0.0,0.0,2
...,...,...,...,...,...,...
35204,490572110001005,2.584262,490572110001005_0,12.0,0.0,0
35205,490572107031007,8.197889,490572107031007_0,31.0,0.0,0
35206,490572106001017,3.022303,490572106001017_0,0.0,0.0,0
35207,490351152111046,59.018298,490351152111046_2,0.0,0.0,2


In [144]:
dfSumByGEOID10 = dfResCBSAdd.groupby([strGEOIDField],as_index=False).agg(NUMBLOCKSPLITS=('BLKSPLITID','count'),BLK_ACRES=('ACRES','sum'),BLK_HH19=('HH19','sum'),BLK_ADDPT=('ADDPT','sum'))
dfSumByGEOID10['SPLIT'] = 0
dfSumByGEOID10.loc[(dfSumByGEOID10['NUMBLOCKSPLITS']>1),'SPLIT'] = 1
dfSumByGEOID10

,GEOID20,NUMBLOCKSPLITS,BLK_ACRES,BLK_HH19,BLK_ADDPT,SPLIT
0,490111251021000,2,1148.054350,0.0,0.0,1
1,490111251021001,2,215.000307,7.0,0.0,1
2,490111251021002,1,3.944387,0.0,0.0,0
3,490111251021003,3,5048.871670,6.0,3.0,1
4,490111251021004,2,132.060814,0.0,0.0,1
...,...,...,...,...,...,...
31773,490572112025022,1,2.542601,11.0,0.0,0
31774,490572112025023,1,6.307278,26.0,0.0,0
31775,490572112025024,1,2.449313,187.0,0.0,0
31776,490572112025025,1,2.235727,0.0,0.0,0


In [145]:
dfResCBSwithTotals = pd.DataFrame.merge(dfResCBSAdd,dfSumByGEOID10,on=strGEOIDField)
dfResCBSwithTotals

,GEOID20,ACRES,BLKSPLITID,HH19,ADDPT,SPLIT_IDX,NUMBLOCKSPLITS,BLK_ACRES,BLK_HH19,BLK_ADDPT,SPLIT
0,490111269024010,3.402761,490111269024010_0,7.0,0.0,0,1,3.402761,7.0,0.0,0
1,490111254152011,4.501047,490111254152011_2,15.0,0.0,2,2,9.362327,30.0,0.0,1
2,490111254152011,4.861280,490111254152011_1,15.0,0.0,1,2,9.362327,30.0,0.0,1
3,490111270024054,0.315008,490111270024054_0,0.0,0.0,0,1,0.315008,0.0,0.0,0
4,490111263033003,317.762203,490111263033003_0,0.0,0.0,0,1,317.762203,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
35204,490351131141013,2.099424,490351131141013_0,0.0,0.0,0,1,2.099424,0.0,0.0,0
35205,490572110001005,2.584262,490572110001005_0,12.0,0.0,0,1,2.584262,12.0,0.0,0
35206,490572107031007,8.197889,490572107031007_0,31.0,0.0,0,1,8.197889,31.0,0.0,0
35207,490572106001017,3.022303,490572106001017_0,0.0,0.0,0,1,3.022303,0.0,0.0,0


In [146]:
dfResCBSwithTotals[dfResCBSwithTotals[strGEOIDField]=='490572101012002']

,GEOID20,ACRES,BLKSPLITID,HH19,ADDPT,SPLIT_IDX,NUMBLOCKSPLITS,BLK_ACRES,BLK_HH19,BLK_ADDPT,SPLIT


In [147]:
dfResCBSwithTotals['PCT_ACRES'] = dfResCBSwithTotals['ACRES' ]/dfResCBSwithTotals['BLK_ACRES']
#dfResCBSwithTotals['PCT_HH10' ] = dfResCBSwithTotals['HH10'  ]/dfResCBSwithTotals['BLK_HH10' ]
dfResCBSwithTotals['PCT_HH19' ] = dfResCBSwithTotals['HH19'  ]/dfResCBSwithTotals['BLK_HH19' ]

#dfResCBSwithTotals.loc[(dfResCBSwithTotals['BLK_HH10']==0),'PCT_HH10'] = dfResCBSwithTotals['PCT_ACRES']
dfResCBSwithTotals.loc[(dfResCBSwithTotals['BLK_HH19']==0),'PCT_HH19'] = dfResCBSwithTotals['PCT_ACRES']

dfResCBSwithTotals

,GEOID20,ACRES,BLKSPLITID,HH19,ADDPT,SPLIT_IDX,NUMBLOCKSPLITS,BLK_ACRES,BLK_HH19,BLK_ADDPT,SPLIT,PCT_ACRES,PCT_HH19
0,490111269024010,3.402761,490111269024010_0,7.0,0.0,0,1,3.402761,7.0,0.0,0,1.000000,1.0
1,490111254152011,4.501047,490111254152011_2,15.0,0.0,2,2,9.362327,30.0,0.0,1,0.480762,0.5
2,490111254152011,4.861280,490111254152011_1,15.0,0.0,1,2,9.362327,30.0,0.0,1,0.519238,0.5
3,490111270024054,0.315008,490111270024054_0,0.0,0.0,0,1,0.315008,0.0,0.0,0,1.000000,1.0
4,490111263033003,317.762203,490111263033003_0,0.0,0.0,0,1,317.762203,0.0,0.0,0,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35204,490351131141013,2.099424,490351131141013_0,0.0,0.0,0,1,2.099424,0.0,0.0,0,1.000000,1.0
35205,490572110001005,2.584262,490572110001005_0,12.0,0.0,0,1,2.584262,12.0,0.0,0,1.000000,1.0
35206,490572107031007,8.197889,490572107031007_0,31.0,0.0,0,1,8.197889,31.0,0.0,0,1.000000,1.0
35207,490572106001017,3.022303,490572106001017_0,0.0,0.0,0,1,3.022303,0.0,0.0,0,1.000000,1.0


In [148]:
dfResCBSwithTotals[['HH19','ADDPT','PCT_ACRES','PCT_HH19','SPLIT','SPLIT_IDX']].sum()

HH19         805272.0
ADDPT          4298.0
PCT_ACRES     31778.0
PCT_HH19      31778.0
SPLIT          5724.0
SPLIT_IDX     11220.0
dtype: float64

In [166]:
# Get GEOID20s

dfResCBSwithTotals_GEOIDs = dfResCBSwithTotals.groupby([strGEOIDField],as_index=False).agg(NUMREC=('HH19','count'))
dfResCBSwithTotals_GEOIDs

,GEOID20,NUMREC
0,490111251021000,2
1,490111251021001,2
2,490111251021002,1
3,490111251021003,3
4,490111251021004,2
...,...,...
31773,490572112025022,1
31774,490572112025023,1
31775,490572112025024,1
31776,490572112025025,1


In [169]:
initial_BG_count = arcpy.management.GetCount("tl_2020_49_tabblock20_UTM_WFModelApprox")
print (initial_BG_count)

dfInitial_GEOIDs = pd.DataFrame.spatial.from_featureclass("tl_2020_49_tabblock20_UTM_WFModelApprox").groupby([strGEOIDField],as_index=False).agg(NUMREC=(strGEOIDField,'count'))
dfInitial_GEOIDs

31775


,GEOID20,NUMREC
0,490111251021000,1
1,490111251021001,1
2,490111251021002,1
3,490111251021003,1
4,490111251021004,1
...,...,...
31770,490572112025022,1
31771,490572112025023,1
31772,490572112025024,1
31773,490572112025025,1


In [172]:
dfOuter = pd.DataFrame.merge(dfResCBSwithTotals_GEOIDs,dfInitial_GEOIDs,on=strGEOIDField,how='outer')
dfOuter[dfOuter.isna().any(axis=1)]

,GEOID20,NUMREC_x,NUMREC_y
29182,490572101011007,1,NaN
29183,490572101011009,1,NaN
29201,490572101013007,1,NaN


In [ ]:
# all three are Weber Boundary - Manually deleted from 

In [150]:
#check for unmatched records

sdfOriginalBG = pd.DataFrame.spatial.from_featureclass("tl_2020_49_tabblock20_UTM_WFModelApprox")

dfSumByGEOID20 = dfResCBSwithTotals.groupby([strGEOIDField]).agg(NUMBLOCKSPLITS=('SPLIT_IDX','max'))

dfCheckMatch = pd.DataFrame.merge(sdfOriginalBG[[strGEOIDField]],dfSumByGEOID20,on=strGEOIDField,how='left')
dfCheckMatch[dfCheckMatch['NUMBLOCKSPLITS'].isna()]

,GEOID20,NUMBLOCKSPLITS


In [152]:
# create a unique name for CSV since ArcGIS Pro seems to have hard time when data changes but filename is the same
strTimeStamp = datetime.now().strftime("%Y%m%d_%H%M%S")
strSplitCSV = "Split_" + strTimeStamp  + ".csv"
print(strSplitCSV)

strSplitCSVTableName = strSplitCSV.replace(".", "_")
print(strSplitCSVTableName)

Split_20220511_161432.csv
Split_20220511_161432_csv


In [153]:
dfResCBSwithTotals[['BLKSPLITID','SPLIT','SPLIT_IDX','HH19','ADDPT','BLK_ACRES','BLK_HH19','BLK_ADDPT','PCT_ACRES','PCT_HH19']].to_csv(strSplitCSV)

In [154]:
# add UTM x and y coordinates of central point
arcpy.management.AddGeometryAttributes("Census_Block_Splits_Final", "CENTROID_INSIDE")

<Result 'Census_Block_Splits_Final'>

In [155]:
#arcpy.management.RemoveJoin("Census_Block_Splits_Final")
arcpy.management.AddJoin("Census_Block_Splits_Final", "BLKSPLITID", strSplitCSV, "BLKSPLITID")

<Result 'Census_Block_Splits_Final'>

In [156]:
arcpy.management.CopyFeatures("Census_Block_Splits_Final", "Census_Block_Splits_Final_wHHSplits")

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\Census Block to TAZ.gdb\\Census_Block_Splits_Final_wHHSplits'>

In [157]:
arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", strGEOIDField, "TEXT", field_length=25)
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", strGEOIDField, "!Census_Block_Splits_Final_" + strGEOIDField + "!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "ACRES", "DOUBLE")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "ACRES", "!" + strSplitCSVTableName + "_BLK_ACRES!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "HH19", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "HH19", "!" + strSplitCSVTableName + "_BLK_HH19!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "SPLIT", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "SPLIT", "!" + strSplitCSVTableName + "_SPLIT!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "SPLIT_IDX", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "SPLIT_IDX", "!" + strSplitCSVTableName + "_SPLIT_IDX!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "BLKSPLITID", "TEXT", field_length=25)
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "BLKSPLITID", "!Census_Block_Splits_Final_BLKSPLITID!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "X_BLKSPLIT", "DOUBLE", field_length=25)
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "X_BLKSPLIT", "!Census_Block_Splits_Final_INSIDE_X!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "Y_BLKSPLIT", "DOUBLE", field_length=25)
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "Y_BLKSPLIT", "!Census_Block_Splits_Final_INSIDE_Y!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "PCT_ACRES", "DOUBLE")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "PCT_ACRES", "!" + strSplitCSVTableName + "_PCT_ACRES!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "PCT_HH19", "DOUBLE")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "PCT_HH19", "!" + strSplitCSVTableName + "_PCT_HH19!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "ACRES_BS", "DOUBLE")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "ACRES_BS", "!Census_Block_Splits_Final_ACRES!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "HH19_BS", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "HH19_BS", "!" + strSplitCSVTableName + "_HH19!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "CO_TAZID", "LONG")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "CO_TAZID", "!Census_Block_Splits_Final_CO_TAZID!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "SUBAREAID", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "SUBAREAID", "1", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "ADDPT", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "ADDPT", "!" + strSplitCSVTableName + "_BLK_ADDPT!", "PYTHON", "")

arcpy.management.AddField("Census_Block_Splits_Final_wHHSplits", "ADDPT_BS", "INTEGER")
arcpy.CalculateField_management("Census_Block_Splits_Final_wHHSplits", "ADDPT_BS", "!" + strSplitCSVTableName + "_ADDPT!", "PYTHON", "")

<Result 'Census_Block_Splits_Final_wHHSplits'>

In [158]:
# delte 
fldlist = arcpy.ListFields("Census_Block_Splits_Final_wHHSplits")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','CO_TAZID',strGEOIDField,'BLKSPLITID','SPLIT','SPLIT_IDX','Shape','Shape_Length','Shape_Area','ACRES','HH10','HH19','ACRES_BS','HH19_BS','ADDPT_BS','PCT_ACRES','PCT_HH19','ADDPT','X_BLKSPLIT','Y_BLKSPLIT','SUBAREAID']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("Census_Block_Splits_Final_wHHSplits",[fld.name])
print("DONE!")

deleting field: Census_Block_Splits_Final_BLKSPLITID
deleting field: Census_Block_Splits_Final_GEOID20
deleting field: Census_Block_Splits_Final_CO_TAZID
deleting field: Census_Block_Splits_Final_ACRES
deleting field: Census_Block_Splits_Final_INSIDE_X
deleting field: Census_Block_Splits_Final_INSIDE_Y
deleting field: Split_20220511_161432_csv_Field1
deleting field: Split_20220511_161432_csv_BLKSPLITID
deleting field: Split_20220511_161432_csv_SPLIT
deleting field: Split_20220511_161432_csv_SPLIT_IDX
deleting field: Split_20220511_161432_csv_HH19
deleting field: Split_20220511_161432_csv_ADDPT
deleting field: Split_20220511_161432_csv_BLK_ACRES
deleting field: Split_20220511_161432_csv_BLK_HH19
deleting field: Split_20220511_161432_csv_BLK_ADDPT
deleting field: Split_20220511_161432_csv_PCT_ACRES
deleting field: Split_20220511_161432_csv_PCT_HH19
DONE!


In [164]:
# manually delete polygons with very small area, acres near 0

# 4 very, very small splits were deleted, hence the new value of 35205
arcpy.management.GetCount("Census_Block_Splits_Final_wHHSplits")

<Result '35205'>

In [160]:
arcpy.management.CopyFeatures("Census_Block_Splits_Final_wHHSplits", "CensusBlockTAZSplitsWithHouseholds_" + strTimeStamp         )
arcpy.management.CopyFeatures("Census_Block_Splits_Final_wHHSplits", "CensusBlockTAZSplitsWithHouseholds_" + strTimeStamp + ".shp")

<Result 'E:\\GitHub\\Census-Block-to-TAZ\\_ArcGIS-Pro-Project\\CensusBlockTAZSplitsWithHouseholds_20220511_161432.shp'>

# One last check

In [161]:
strLayerName = "CensusBlockTAZSplitsWithHouseholds_" + strTimeStamp + ".shp"

initial_count = arcpy.management.GetCount(strLayerName)
print (initial_count)

# union with self to see if overlapping polygons
arcpy.analysis.Union([strLayerName, strLayerName], "tempSelfUnion")
selfunion_count = arcpy.management.GetCount("tempSelfUnion")
print (selfunion_count)

#arcpy.management.Delete("tempSelfUnion")

# set layer definition for TAZ
for lyr in m.listLayers("tempSelfUnion"):
    lyrAddress = lyr
    if lyr.supports("DEFINITIONQUERY"):
        lyr.definitionQuery = "BLKSPLITID <> BLKSPLITID_1"


35205
35205
